# Resilient Distributed Datasets (RDDs)

*Resilient Distributed Dataset* (RDD) est une collection d'objets immuables et distribués. Les RDD sont résilients ou tolérants aux pannes. Les collections d'objets partitionnés sont réparties dans un cluster, stockées en mémoire ou sur disque. Les RDD sont construits et manipulés grâce à un ensemble diversifié de transformations parallèles (*map*, *filter*, *join*) et d'actions (*count*, *collect*, *save*)

Un RDD peut être créé de plusieurs manières:
* Paralléliser une collection
* Lire des données à partir d'une source externe
* Transformation d'un RDD existant
* API de streaming

## RDD  Operations

###  Transformation (Lazy evaluation)
Les transformations créent de nouvelles dataset à partir d'une dataset existante. Par exemple, *map* est une transformation qui passe chaque élément de l'ensemble de données via une fonction et renvoie un nouveau RDD représentant les résultats. Toutes les transformations dans Spark sont *lazy*, c'est à dire elles ne calculent pas le résultats automatiquement. Les transformations ne sont calculées que lorsqu'une action nécessite qu'un résultat soit renvoyé.  

**Exemple de transformations**

*map(),  flatMap(), filter(),  distinct(), intersection(), cartesian(), groupByKey(), coalesce(), mapPartitions(), reduceByKey(), repartition(), sortByKey(), partitionBy(), sample(), ...*  

### Actions
Les actions permettent de returner la valeur calculer sur le dataset vers le driver. Par exemple, *reduce* est une action qui agrège tous les éléments du RDD en utilisant une fonction et renvoie le résultat final vers le driver.

**Exemple d'actions**
*reduce(), collect(), count(), first(), take() countByKey(), takeSample(), foreach(), takeOrdered(), saveAsTextFile(), saveAsSequenceFile() ...*

![](images/rdd.png)

**Note** : Dans ce chapitre nous allons faire une introduction des RDD pour vous permettre de comprendre comment fonctionne Spark built-in. Depuis les récentes versions de Spark,les RDD sont désormais considérés dans API de bas niveau.

#### Parralleliser un collection

In [1]:
val rdd = sc.parallelize(Seq(0,1,2,3,4,5,6,7,8,9))

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.115.226:4041
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1659193087151)
SparkSession available as 'spark'


rdd: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[0] at parallelize at <console>:25


In [2]:
rdd

res0: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[0] at parallelize at <console>:25


In [3]:
rdd.take(10)

res1: Array[Int] = Array(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)


In [4]:
rdd.take(5).foreach(println)

0
1
2
3
4


In [7]:
def even(x : Int) = x % 2 == 0

even: (x: Int)Boolean


In [3]:
even(2)

res0: Boolean = true


application d'une transformation `filter` sur le RDD retourne une nouvelle RDD 

In [8]:
val rdd_filter = rdd.filter(x => x % 2 == 0)

rdd_filter: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[2] at filter at <console>:26


In [9]:
val rdd_filter = rdd.filter(even)

rdd_filter: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[3] at filter at <console>:28


In [10]:
rdd_filter.collect()

res3: Array[Int] = Array(0, 2, 4, 6, 8)


L'action `take` sur le RDD permet d'extraire les 5 premiers éléments

In [13]:
rdd.filter(x => x % 2 == 0).take(5)

res4: Array[Int] = Array(0, 2, 4, 6, 8)


In [17]:
val rdd2 = sc.parallelize(Seq(1, 2, 4,5, 9, 11))

rdd2: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[11] at parallelize at <console>:25


In [18]:
rdd2.map(x => x * x).filter(x => x % 2 == 0).collect()

res7: Array[Int] = Array(4, 16)


In [19]:
rdd2.map(x => x * x).filter(x => x % 2 == 0).reduce((a, b) => a+b)

res8: Int = 20


L'action `collect` sur le RDD permet d'extraire la liste de tous les éléments. Eviter d'utiliser RDD lorsqu'on a une large volume de données

In [20]:
rdd_filter.collect()

res9: Array[Int] = Array(0, 2, 4, 6, 8)


la fonction `map` constitue une transformation et permet d'appliquer une fonction en input sur le RDD

In [9]:
val rdd_map = rdd_filter.map(x => x*x)

rdd_map: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[3] at map at <console>:26


In [10]:
rdd_map.take(10)

res6: Array[Int] = Array(0, 4, 16, 36, 64)


L'action de la fonction `reduce` permet d'appliquer une fonction d'aggregation somme sur le dataset.

In [11]:
val result = rdd_map.reduce((a, b) => a+b)

result: Int = 120


Nous pouvons factoriser le code en créant un *pipeline* de transformation suivi d'une action

In [12]:
val sum_even_sqr =  rdd.filter(x => x % 2 == 0).map( x => x*x).map(x => x*x*x).reduce((x,y) => x + y)

sum_even_sqr: Int = 312960


### Lecture d'un fichier texte

`sc.textFile` permet lire un fichier texte et le transforme en RDD.

In [11]:
val texte= sc.textFile("datasets/senegal.txt")

texte: org.apache.spark.rdd.RDD[String] = datasets/senegal.txt MapPartitionsRDD[5] at textFile at <console>:25


Nombre de ligne

In [22]:
texte.count

res10: Long = 5


In [23]:
texte.first

res11: String = "Le Sénégal est un pays situé sur la côte ouest de l'Afrique et doté d'un héritage colonial français et de nombreuses attractions naturelles. "


### Preprocessing sur un fichier texte

convertion du texte en Majuscule

In [16]:
val fistLines = texte.map(line => line.toUpperCase()).take(2)

fistLines: Array[String] = Array("LE SÉNÉGAL EST UN PAYS SITUÉ SUR LA CÔTE OUEST DE L'AFRIQUE ET DOTÉ D'UN HÉRITAGE COLONIAL FRANÇAIS ET DE NOMBREUSES ATTRACTIONS NATURELLES. ", "DAKAR, LA CAPITALE, COMPREND LE QUARTIER HISTORIQUE DE LA MÉDINA ET LE CÉLÈBRE MUSÉE THÉODORE MONOD, EXPOSANT DES ŒUVRES D'ART AFRICAIN. ")


In [17]:
fistLines(0)

res9: String = "LE SÉNÉGAL EST UN PAYS SITUÉ SUR LA CÔTE OUEST DE L'AFRIQUE ET DOTÉ D'UN HÉRITAGE COLONIAL FRANÇAIS ET DE NOMBREUSES ATTRACTIONS NATURELLES. "


filtrage des ligne commençant par L

In [18]:
texte.filter(line => line.startsWith("L")).take(4).foreach(println)

Le Sénégal est un pays situé sur la côte ouest de l'Afrique et doté d'un héritage colonial français et de nombreuses attractions naturelles. 
Le Sénégal est un pays dont le président est démocratiquement élu au suffrage universel directe.


Factorisation du code

In [19]:
texte.map(line => line.toUpperCase()).filter(line => line.startsWith("L")).take(4).foreach(println)

LE SÉNÉGAL EST UN PAYS SITUÉ SUR LA CÔTE OUEST DE L'AFRIQUE ET DOTÉ D'UN HÉRITAGE COLONIAL FRANÇAIS ET DE NOMBREUSES ATTRACTIONS NATURELLES. 
LE SÉNÉGAL EST UN PAYS DONT LE PRÉSIDENT EST DÉMOCRATIQUEMENT ÉLU AU SUFFRAGE UNIVERSEL DIRECTE.


`flatMap` applique une transformation .split sur le RDD collections pour obtenir la liste des mot contenus dans le texte 

In [23]:
texte.collect()

res15: Array[String] = Array("Le Sénégal est un pays situé sur la côte ouest de l'Afrique et doté d'un héritage colonial français et de nombreuses attractions naturelles. ", "Dakar, la capitale, comprend le quartier historique de la Médina et le célèbre musée Théodore Monod, exposant des œuvres d'art africain. ", "Elle est également réputée pour sa vie nocturne, centrée sur la musique mbalax, originaire du Sénégal. ", "Saint-Louis, ancienne capitale de l'Afrique-Occidentale française, abrite une vieille ville à l'architecture coloniale. ", Le Sénégal est un pays dont le président est démocratiquement élu au suffrage universel directe.)


In [24]:
"Le Sénégal est un pays situé sur la côte ouest de l'Afrique".split(' ')

res16: Array[String] = Array(Le, Sénégal, est, un, pays, situé, sur, la, côte, ouest, de, l'Afrique)


In [25]:
texte.flatMap(line => line.split(' ')).

res17: Array[String] = Array(Le, Sénégal, est, un, pays, situé, sur, la, côte, ouest, de, l'Afrique, et, doté, d'un, héritage, colonial, français, et, de, nombreuses, attractions, naturelles., Dakar,, la, capitale,, comprend, le, quartier, historique, de, la, Médina, et, le, célèbre, musée, Théodore, Monod,, exposant, des, œuvres, d'art, africain., Elle, est, également, réputée, pour, sa, vie, nocturne,, centrée, sur, la, musique, mbalax,, originaire, du, Sénégal., Saint-Louis,, ancienne, capitale, de, l'Afrique-Occidentale, française,, abrite, une, vieille, ville, à, l'architecture, coloniale., Le, Sénégal, est, un, pays, dont, le, président, est, démocratiquement, élu, au, suffrage, universel, directe.)


In [20]:
val rdd = texte.flatMap(line => line.split(' ')).distinct()

rdd: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[16] at distinct at <console>:26


In [21]:
rdd.take(10).foreach(println)

à
sur
française,
directe.
pour
des
ouest
comprend
d'un
coloniale.


### zip

In [22]:
val capitales = sc.parallelize(Seq("Abuja","Accra", "Addis-Abeba", "Alger", "Dakar", "Banjul", 
                                   "Antananarivo", "Bamako", "Bissau","Caire"))

capitales: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[17] at parallelize at <console>:25


In [23]:
capitales.take(5)

res13: Array[String] = Array(Abuja, Accra, Addis-Abeba, Alger, Dakar)


In [24]:
val pays =  sc.parallelize(Seq("Nigeria", "Ghana", "Éthiopie", "Algérie", "Sénégal", "Gambie", 
                               "Madagascar","Mali","Guinée-Bissau", "Égypte"))

pays: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[18] at parallelize at <console>:25


`.zip` permet des generer couples entre chaque pays et son capitale.

In [25]:
val pays_capitales = pays.zip(capitales)

pays_capitales: org.apache.spark.rdd.RDD[(String, String)] = ZippedPartitionsRDD2[19] at zip at <console>:28


In [26]:
 pays_capitales.collect().foreach(println)

(Nigeria,Abuja)
(Ghana,Accra)
(Éthiopie,Addis-Abeba)
(Algérie,Alger)
(Sénégal,Dakar)
(Gambie,Banjul)
(Madagascar,Antananarivo)
(Mali,Bamako)
(Guinée-Bissau,Bissau)
(Égypte,Caire)


### Le fameux wordcount avec Spark (RDD)

In [13]:
val rdd_figaro =  sc.textFile("datasets/ancien_figaro.txt")

rdd_figaro: org.apache.spark.rdd.RDD[String] = datasets/ancien_figaro.txt MapPartitionsRDD[9] at textFile at <console>:25


In [16]:
val rdd_flatmap = rdd_figaro.flatMap(line => line.split(" "))

rdd_flatmap: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[12] at flatMap at <console>:26


In [17]:
val rdd_map = rdd_flatmap.map(w => (w, 1))

rdd_map: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[13] at map at <console>:26


In [18]:
val wordcounts = rdd_map.reduceByKey((w1, w2) => w1 + w2)

wordcounts: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[14] at reduceByKey at <console>:26


In [19]:
wordcounts

res4: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[14] at reduceByKey at <console>:26


In [20]:
wordcounts.take(10).foreach(println)

(journée,,1)
(Ah!,19)
(apostrophes,2)
(destitué;,1)
(budget!,2)
(souvent,22)
(combinaison,,1)
(épigrammes.,2)
(plaisir,,2)
(resteront,1)


Forme condensée

In [21]:
val wordcounts = rdd_figaro.flatMap(_.split(" ")).map((_, 1)).reduceByKey(_+_)   

wordcounts: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[17] at reduceByKey at <console>:26


In [22]:
wordcounts.take(10).foreach(println)

(journée,,1)
(Ah!,19)
(apostrophes,2)
(destitué;,1)
(budget!,2)
(souvent,22)
(combinaison,,1)
(épigrammes.,2)
(plaisir,,2)
(resteront,1)


### Exercice  

Reprendre le wordcount en supprimant les ponctuations, rendre le texte en minuscule, et ordonner le resultat en ordre décroissant des mots.